# 삼성 DS AI Expert Program

## Sentence Parsing Assignment

담당 조교 : 문승준 (june1212@kaist.ac.kr)

실습 일시: 2019년 10월 8일 (화), 13:30 - 17:30

## Introduction

본 실습에서는 Tensorflow와 Keras를 이용하여, Sentence Parsing을 직접 진행해 볼 것입니다. Sentence Parsing은 문장 구성 요소 사이의 연관성을 찾아서 기계의 자연어 이해를 돕는 task로, 실습을 통해 직접 구현해보겠습니다. 이번 실습에서 사용되는 자연어 처리 모델은 Seq2Seq와 Transformer입니다.

## 1. Preprocess

Training을 하기에 앞서, pretrain된 word embedding과 dataset을 load하겠습니다.

In [ ]:
from pathlib import Path
from collections import Counter

### Dataset 불러오기

Train dataset과 Test dataset을 불러옵니다. Demo이기 때문에 실제 dataset보다 훨씬 작은 크기로 구축되어 있습니다.

In [ ]:
!wget https://raw.githubusercontent.com/hankook/Samsung-AI-KAIST/master/191008_Sentence_Parsing/train.tsv
!wget https://raw.githubusercontent.com/hankook/Samsung-AI-KAIST/master/191008_Sentence_Parsing/test.tsv

source와 target에 포함되는 vocab dataset을 불러옵니다.

In [ ]:
!mkdir vocab
!wget -P ./vocab https://raw.githubusercontent.com/hankook/Samsung-AI-KAIST/master/191008_Sentence_Parsing/vocab/source.txt
!wget -P ./vocab https://raw.githubusercontent.com/hankook/Samsung-AI-KAIST/master/191008_Sentence_Parsing/vocab/target.txt
    

Pre-train된 word embedding은 glove.300d를 사용합니다.

In [ ]:
!wget -P ./vocab http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip ./vocab/glove.840B.300d.zip -d ./vocab/

In [ ]:
enc_counter = Counter()
dec_counter = Counter()

with open('./train.tsv') as f:
  for line in f:
    line = line.rstrip()
    text_raw, text_tokenized, label = line.split('\t')
    enc_counter.update(text_tokenized.lower().split())
    dec_counter.update(label.replace('[', '[ ').lower().split())

with open('./vocab/source.txt', 'w') as f:
  f.write('<pad>\n')
  for (w, freq) in enc_counter.most_common():
    f.write(w+'\n')
    
with open('./vocab/target.txt', 'w') as f:
  f.write('<pad>\n')
  f.write('<start>\n')
  f.write('<end>\n')
  for (w, freq) in dec_counter.most_common():
    f.write(w+'\n')

### Pretrain 된 Embedding 만들기

glove.840B.300d.txt 파일을 이용해 word.npy 파일을 만들어줍니다.

In [ ]:
import numpy as np

word2idx = {}
with open('./vocab/target.txt', encoding='utf-8') as f:
  for i, line in enumerate(f):
    line = line.rstrip()
    word2idx[line] = i

In [ ]:
embedding = np.zeros((len(word2idx)+1, 300)) # + 1 은 unknown word를 포함하기 위해서 더해준다.

with open('./vocab/glove.840B.300d.txt', encoding='utf-8') as f:
  count = 0
  for i, line in enumerate(f):
    if i % 100000 == 0:
      print('- At line {}'.format(i))
    line = line.rstrip()
    sp = line.split(' ')
    word, vec = sp[0], sp[1:]
    if word in word2idx:
      count += 1
      embedding[word2idx[word]] = np.asarray(vec, dtype='float32')
      
print("[%d / %d] words have found pre-trained values"%(count, len(word2idx)))
np.save('./vocab/word.npy', embedding)
print('Saved ./vocab/word.npy')